In this homework, we'll deploy the Straight vs Curly Hair Type model we trained in the
[previous homework](../08-deep-learning/homework.md).

Download the model files from here:

* https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
* https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx

In [1]:
PREFIX = "https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
DATA_URL = f"{PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL = f"{PREFIX}/hair_classifier_v1.onnx"

import subprocess
subprocess.run(["wget", DATA_URL])
subprocess.run(["wget", MODEL_URL])

CompletedProcess(args=['wget', 'https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx'], returncode=0)

## Question 1

To be able to use this model, we need to know the name of the input and output nodes.

What's the name of the output:

* `output`
* `sigmoid`
* `softmax`
* `prediction`

In [3]:
import onnxruntime as ort

onnx_model_path = "hair_classifier_v1.onnx"
session = ort.InferenceSession(onnx_model_path, providers=["CPUExecutionProvider"])

inputs = session.get_inputs()
outputs = session.get_outputs()

input_name = inputs[0].name
output_name = outputs[0].name
print(f"Input name: {input_name}")
print(f"Output name: {output_name}")

Input name: input
Output name: output


The name of the output nodes: `output`

## Preparing the image


In [4]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

## Question 2: Target size

Let's download and resize this image:

https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

Based on the previous homework, what should be the target size for the image?

* 64x64
* 128x128
* 200x200
* 256x256

The shape for input of the model in previous homework (i.e. homework for 08-deep-learning) is (3, 200, 200).

Hence, the target size for the image should be `200x200`.

## Question 3

Now we need to turn the image into numpy array and pre-process it.

> Tip: Check the previous homework. What was the pre-processing
> we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

* -10.73
* -1.073
* 1.073
* 10.73

In [5]:
url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
downloaded_img = download_image(url)
target_size = (200, 200)
resized_img = prepare_image(downloaded_img, target_size)

In [8]:
import os
from torchvision import transforms
from torch.utils.data import Dataset # Import Dataset
import numpy as np

# The HairStyleDataset is not needed for processing a single image as required by Question 3.
# The task is to preprocess the 'resized_img' obtained in the previous step.
# class HairStyleDataset(Dataset):
#     def __init__(self, data_dir, transform=None):
#         self.data_dir = data_dir
#         self.transform = transform
#         self.image_paths = []
#         self.labels = []
#         self.classes = sorted(os.listdir(data_dir))
#         self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

#         for label_name in self.classes:
#             label_dir = os.path.join(data_dir, label_name)
#             for img_name in os.listdir(label_dir):
#                 self.image_paths.append(os.path.join(label_dir, img_name))
#                 self.labels.append(self.class_to_idx[label_name])

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         img_path = self.image_paths[idx]
#         image = Image.open(img_path).convert('RGB')
#         label = self.labels[idx]

#         if self.transform:
#             image = self.transform(image)

#         return image, label

preprocessing = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

# Apply the preprocessing directly to the resized_img
preprocessed_img_tensor = preprocessing(resized_img)

# Convert to numpy array to inspect pixel values
# The tensor has shape (C, H, W). We need to access the R channel of the first pixel (0,0).
# For the first pixel, the R channel is at preprocessed_img_tensor[0, 0, 0].
first_pixel_r_channel = preprocessed_img_tensor[0, 0, 0].item()

print(f"Value in the first pixel, R channel: {first_pixel_r_channel:.3f}")

# The HairStyleDataset and DataLoader are not needed for this question, commenting them out.
# train_dataset = HairStyleDataset(
#     data_dir='./yf_dokzqy3vcritme8ggnzqlvwa.jpeg',
#     transform=preprocessing
# )

# from torch.utils.data import DataLoader

# train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)

Value in the first pixel, R channel: -1.073


The value in the first pixel, the R channel: -1.073

## Question 4

Now let's apply this model to this image. What's the output of the model?

* 0.09
* 0.49
* 0.69
* 0.89

In [11]:
X = preprocessed_img_tensor
# Convert the PyTorch tensor to a NumPy array and add a batch dimension
X_np = X.unsqueeze(0).numpy()
result = session.run([output_name], {input_name: X_np})
# The result is a list containing an array, so we access the first element
# and then the first element of that array if it's a single prediction.
print(result[0][0])

[0.09156641]


The output of the model: 0.09

## Prepare the lambda code 

Now you need to copy all the code into a separate python file. You will 
need to use this file for the next two questions.

Tip: you can test this file locally with `ipython` or Jupyter Notebook 
by importing the file and invoking the function from this file.  


## Docker 

For the next two questions, we'll use a Docker image that we already 
prepared. This is the Dockerfile that we used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.13

COPY hair_classifier_empty.onnx.data .
COPY hair_classifier_empty.onnx .
```

Note that it uses Python 3.13.

The docker image is published to [`agrigorev/model-2025-hairstyle:v1`](https://hub.docker.com/r/agrigorev/model-2025-hairstyle).

A few notes:

* The image already contains a model and it's not the same model
  as the one we used for questions 1-4.

## Question 5

Download the base image `agrigorev/model-2025-hairstyle:v1`. You can do it with [`docker pull`](https://docs.docker.com/engine/reference/commandline/pull/).

So what's the size of this base image?

* 88 Mb
* 208 Mb
* 608 Mb
* 1208 Mb

You can get this information when running `docker images` - it'll be in the "SIZE" column.

The size of this base image: 608 MB

## Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included. 
The name of the file with the model is `hair_classifier_empty.onnx` and it's 
in the current workdir in the image (see the Dockerfile above for the 
reference). 
The provided model requires the same preprocessing for images regarding target size and rescaling the value range than used in homework 8.

Now run the container locally.

Score this image: https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

What's the output from the model?

* -1.0
* -0.10
* 0.10
* 1.0